In [5]:
import random
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [40]:
INPUTSHAPE = (1, 41, 41)
SEQUENTIALINPUTSHAPE = (1669, 1)
NUMCONDITIONS = 10

In [41]:
class ConvolutionPkmnPlayer(keras.Sequential):
    def __init__(self, _input_shape = INPUTSHAPE, _num_conditions = NUMCONDITIONS, _kernel_size = (12,12), _dilations = 10, _num_hidden_units = 256):
        print("INITIALIZING ACTOR")
        super(ConvolutionPkmnPlayer, self).__init__([
            #Normalize the data to reduce sensitivity to individual values and increase training stability
            keras.layers.BatchNormalization(name = "Normalization", input_shape = _input_shape),
            #Initialize our Conv2d layer with a kernel of 10, 10 and 10 dilations to generate activations for next layer
            keras.layers.Conv2D(_dilations, _kernel_size, activation = 'relu', padding = "same", name = "FirstConv2d"),
            #Begin our sequence of Alternating Conv2d Layers and Max pooling layers to prevent overfitting to various locations within the Data
            keras.layers.Conv2D(_dilations * 2, _kernel_size, activation = 'relu', padding = "same", name = "SecConv2d"),
            keras.layers.MaxPooling2D(padding = "same", name = "FirstMaxPooling"),
            keras.layers.Conv2D(_dilations * 2, _kernel_size, activation = 'relu', padding = "same", name = "ThirdConv2d"),
            keras.layers.MaxPooling2D(padding = "same", name = "SecMaxPooling"),
            #Dropout to prevent overfitting
            keras.layers.Dropout(0.2, name = "Dropout"),
            keras.layers.Conv2D(_dilations * 2, _kernel_size, activation = 'relu', padding = "same", name = "FourthConv2d"),
            #Flattening to pass the activations to Dense Layer
            keras.layers.Flatten(name = "Flatten"),
            #Dense Layer to increase depth and node count
            keras.layers.Dense(_num_hidden_units, activation = 'relu', name = "Dense"),
            #Softmax output layer
            keras.layers.Dense(_num_conditions, activation = 'softmax')
        ])
    def call(self, x, training = True):
        x = x.reshape(INPUTSHAPE)
        for layer in self.layers:
            x = layer(x)
        return x

In [46]:
class SequentialPkmnPlayer(keras.Sequential):
    def __init__(self, _input_shape = SEQUENTIALINPUTSHAPE[0], num_units = 128, _num_conditions = NUMCONDITIONS):
        super(SequentialPkmnPlayer, self).__init__([
            keras.layers.Dense(_input_shape, activation = 'relu'),
            keras.layers.Dense(num_units * 2, activation = 'relu'),
            keras.layers.Dense(num_units * 2, activation = 'relu'),
            keras.layers.Dense(num_units * 2, activation = 'relu'),
            keras.layers.Dense(num_units, activation = 'relu'),
            keras.layers.Dense(num_units, activation = 'relu'),
            keras.layers.Dense(_num_conditions, activation = 'softmax'),
            keras.layers.Dense(1, activation = 'tanh', name = 'critic')
        ])
    def call(self, x, training = True):
        #pass input through every Layer
        for i in range(len(self.layers) - 2):
            x = self.layers[i](x)
        #Actor, Critic
        return self.layers[-2](x), self.layers[-1](x)

In [11]:
class ResidualPkmnPlayer:
    def relu_bn(inputs: tf.Tensor) -> tf.Tensor:
        relu = keras.layers.ReLU()(inputs)
        bn = keras.layers.BatchNormalization()(relu)
        return bn
    def residual_block(x: tf.Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> tf.Tensor:
        y = keras.layers.Conv2D(kernel_size=kernel_size,
                   strides= (1 if not downsample else 2),
                   filters=filters,
                   padding="same")(x)
        y = ResidualPkmnPlayer.relu_bn(y)
        y = keras.layers.Conv2D(kernel_size=kernel_size,
                   strides=1,
                   filters=filters,
                   padding="same")(y)

        if downsample:
            x = keras.layers.Conv2D(kernel_size=1,
                       strides=2,
                       filters=filters,
                       padding="same")(x)
        out = keras.layers.Add()([x, y])
        out = ResidualPkmnPlayer.relu_bn(out)
        return out
    def getModel():
        inputs = keras.Input(shape=(INPUTSHAPE))
        num_filters = 64

        t = keras.layers.BatchNormalization()(inputs)
        t = keras.layers.Conv2D(kernel_size=3,
                   strides=1,
                   filters=num_filters,
                   padding="same")(t)
        t = ResidualPkmnPlayer.relu_bn(t)

        num_blocks_list = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
        for i in range(len(num_blocks_list)):
            num_blocks = num_blocks_list[i]
            for j in range(num_blocks):
                t = ResidualPkmnPlayer.residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
            num_filters *= 2

        t = keras.layers.AveragePooling2D(4)(t)
        t = keras.layers.Flatten()(t)
        outputs = keras.layers.Dense(1, activation='linear')(t)

        model = Model(inputs, outputs)

        return model

In [14]:
class SequentialPkmnCritic(keras.Sequential):
    """Combined actor-critic network."""

    def __init__(self):

        super(SequentialPkmnCritic, self).__init__([
        #keras.layers.Flatten(input_shape = (1, SEQUENTIALINPUTSHAPE[0])),
        keras.layers.Dense(SEQUENTIALINPUTSHAPE[0], activation = 'tanh'),
        keras.layers.Dense(1, activation = 'tanh', name = 'critic')])
        

In [45]:
def selectAction(action_probs, numActions = 10):
    selector = random.uniform(0, 1)
    prob = float(0)
    for i in range(numActions):
        prob += float(action_probs[0, i])
        if selector <= prob:
            return tf.Variable(i, dtype = tf.int32)
    return tf.Variable(numActions - 1, dtype = tf.int32)

NameError: name 'SEQUENTIALINPUTSHAPE' is not defined